In [11]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [12]:
x = np.load('data/EEG_all_epochs.npy')
y = np.load('data/EEG_contaminated_with_EOG/EEG_contaminated_with_EOG_1db.npy')

# Preprocesamiento de los datos en preparacion para la LSTM

In [13]:
#Normalizacion de los datos
scaler = StandardScaler()
x_norm = scaler.fit_transform(x)
y_norm = scaler.fit_transform(y)

#Reshape de los datos
timesteps = 100
n_samples, n_features = x_norm.shape
n_sequences = n_samples // timesteps
x_reshaped = np.reshape(x_norm[:n_sequences * timesteps, :], (n_sequences, timesteps, n_features))

n_samples, n_features = y_norm.shape
n_sequences = n_samples // timesteps
y_reshaped = np.reshape(y_norm[:n_sequences * timesteps, :], (n_sequences, timesteps, n_features))

print(x_reshaped.shape)
print(y_reshaped.shape)

(45, 100, 512)
(45, 100, 512)


In [14]:
x_train, x_test, y_train, y_test = train_test_split(x_reshaped, y_reshaped, test_size=0.2, random_state=42)

In [15]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(timesteps, n_features)))
model.add(LSTM(32, return_sequences=False))
model.add(Dense(n_features))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=50, validation_data=(x_test, y_test))

In [ ]:
score = model.evaluate(x_test, y_test)